In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.models as models
import numpy as np
import random
from itertools import chain, combinations
from sklearn.metrics import f1_score, average_precision_score
from sklearn.model_selection import train_test_split
from pathlib import Path
import threading
from multiprocessing import Manager
import gc

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cuda:0


In [ ]:
# normalizing our spectrogram using imagenet stats
mean = torch.tensor([0.485, 0.456, 0.406]).view(1, 3, 1, 1)
std = torch.tensor([0.229, 0.224, 0.225]).view(1, 3, 1, 1)

In [ ]:
# Appling frequency and time masking to spectrograms

class SpecAugment:
    """Applies frequency and time masking to spectrograms"""
    def __init__(self, freq_mask_max=0.15, time_mask_max=0.20):
        self.freq_mask_max = freq_mask_max
        self.time_mask_max = time_mask_max
    
    def __call__(self, spec):
        spec = spec.clone()
        n_mels, n_time = spec.shape
        
        # Frequency masking
        if self.freq_mask_max > 0:
            max_freq_bands = int(n_mels * self.freq_mask_max)
            freq_bands = random.randint(1, max_freq_bands)
            freq_start = random.randint(0, n_mels - freq_bands)
            spec[freq_start:freq_start+freq_bands, :] = spec.min()
        
        # Time masking
        if self.time_mask_max > 0:
            max_time_steps = int(n_time * self.time_mask_max)
            time_steps = random.randint(1, max_time_steps)
            time_start = random.randint(0, n_time - time_steps)
            spec[:, time_start:time_start+time_steps] = spec.min()
        
        return spec
    
# It Mixes two audio samples with adjustable ratio

class AudioMixer:
    
    def __init__(self, alpha=0.5, mix_prob=0.5, min_mix_ratio=0.3):
        self.alpha = alpha
        self.mix_prob = mix_prob
        self.min_mix_ratio = min_mix_ratio

    def __call__(self, spec1, label1, specsList, labelsList):
        if random.random() > self.mix_prob:
            return spec1, label1
        idx2 = random.randint(0, specsList.shape[0] - 1)
        spec2 = specsList[idx2]
        label2 = labelsList[idx2]
        if spec1.shape != spec2.shape:
            raise ValueError("Spectrograms is somehow not in same shape")
        lam = np.random.beta(self.alpha, self.alpha)
        lam = lam * 1.0 * (1.0 - 2 * self.min_mix_ratio) + self.min_mix_ratio
        mixed_spec = lam * spec1 + (1 - lam) * spec2
        mixed_label = torch.clamp(label1 + label2, 0, 1)
        return mixed_spec, mixed_label
    
spec_augment = SpecAugment()
audio_mixer = AudioMixer()

def batch_augment(specsList, labelsList):
    augmented_specs = []
    augmented_labels = []
    N = specsList.shape[0]
    for i in range(N):
        spec = specsList[i]
        label = labelsList[i]
        spec, label = audio_mixer(spec, label, specsList, labelsList)
        spec = spec_augment(spec)
        augmented_specs.append(spec)
        augmented_labels.append(label)
    augmented_specs = torch.stack(augmented_specs)
    augmented_labels = torch.stack(augmented_labels)
    augmented_specs = augmented_specs.unsqueeze(1).expand(-1, 3, -1, -1)
    augmented_specs = (augmented_specs - mean) / std
    return augmented_specs, augmented_labels

Handling Data

In [ ]:
class ShardedEpisodicDataset():
    def __init__(self, data_paths, meta_path, min_n_way, try_k_shot, try_n_query, split="train",
                 test_split=0.2, val_split=0.2, split_seed=33, min_examples_per_class=15):
        self.data_paths = data_paths
        self.min_n_way = min_n_way
        self.try_k_shot = try_k_shot
        self.try_n_query = try_n_query

        # load metadata
        meta = torch.load(meta_path)
        self.rel_idxs = meta['rel_idxs']
        self.n_examples = self.rel_idxs[-1] + 1
        meta_class_idxs = meta['class_idxs']
        self.n_classes = len(meta_class_idxs)

        # Get all class indices
        all_class_idxs = [[i.item() for i in meta_class_idxs[c]] for c in range(self.n_classes)]

        # Filter classes with sufficient examples
        valid_classes_for_split = []
        for c in range(self.n_classes):
            if len(all_class_idxs[c]) >= min_examples_per_class:
                valid_classes_for_split.append(c)

        print(f"Classes with >= {min_examples_per_class} examples: {len(valid_classes_for_split)}/{self.n_classes}")

        # CORRECTED: Split SAMPLES within each class, not classes themselves
        # All splits see all classes, but with different samples
        self.class_idxs = [[] for _ in range(self.n_classes)]

        np.random.seed(split_seed)

        for c in valid_classes_for_split:
            class_samples = all_class_idxs[c]

            if len(class_samples) < min_examples_per_class:
                continue

            # Shuffle samples within this class
            shuffled_samples = np.random.permutation(class_samples)

            # Split samples within this class
            n_test = max(1, int(len(shuffled_samples) * test_split))
            n_val = max(1, int(len(shuffled_samples) * val_split))
            n_train = len(shuffled_samples) - n_test - n_val

            # Ensure we have enough samples for episodes
            min_needed = self.try_k_shot + self.try_n_query
            if n_train < min_needed or n_val < min_needed or n_test < min_needed:
                # If not enough samples, give more to each split
                samples_per_split = len(shuffled_samples) // 3
                if samples_per_split < min_needed:
                    # Use all samples for all splits (with overlap) if really limited
                    train_samples = shuffled_samples.tolist()
                    val_samples = shuffled_samples.tolist()
                    test_samples = shuffled_samples.tolist()
                else:
                    train_samples = shuffled_samples[:samples_per_split].tolist()
                    val_samples = shuffled_samples[samples_per_split:2*samples_per_split].tolist()
                    test_samples = shuffled_samples[2*samples_per_split:].tolist()
            else:
                train_samples = shuffled_samples[:n_train].tolist()
                val_samples = shuffled_samples[n_train:n_train+n_val].tolist()
                test_samples = shuffled_samples[n_train+n_val:].tolist()

            # Assign samples based on split
            if split == "train":
                self.class_idxs[c] = train_samples
            elif split == "val":
                self.class_idxs[c] = val_samples
            elif split == "test":
                self.class_idxs[c] = test_samples
            else:
                raise ValueError("Split must be 'train', 'val', or 'test'")

        # Valid classes for episodes
        self.valid_classes = [
            c for c, indices in enumerate(self.class_idxs)
            if len(indices) >= self.try_k_shot + self.try_n_query
        ]

        print(f"{len(self.valid_classes)} classes available for {split} episodes")

        if len(self.valid_classes) < self.min_n_way:
            raise ValueError(f"Not enough valid classes for {split}. Need {self.min_n_way}, have {len(self.valid_classes)}")

        # cache management
        self.cache_size = 1
        self.manager = Manager()
        self.cache = self.manager.dict()
        self.lock = threading.Lock()

    def _get_idxs(self, idx):
        # binary search on shard to find shard
        low, high = 0, len(self.rel_idxs) - 1
        while (low < high):
            mid = (low + high) // 2
            if self.rel_idxs[mid] < idx:
                low = mid + 1
            else:
                high = mid
        shard_idx = low

        if (shard_idx == 0):
            abs_idx = idx
        else:
            abs_idx = idx - self.rel_idxs[shard_idx - 1] - 1
        return shard_idx, abs_idx

    def _load_shard(self, shard_idx):
        shard_path = self.data_paths[shard_idx]

        with self.lock:
            # Check if shard is already in cache
            if shard_path in self.cache:
                return self.cache[shard_path]

            # If cache is full, remove one item
            if len(self.cache) >= self.cache_size:
                # More predictable eviction strategy
                oldest_key = next(iter(self.cache))
                self.cache.pop(oldest_key)

            # Load the shard and store in cache
            shard = torch.load(shard_path)
            self.cache[shard_path] = shard

        return shard

    def getEpisode(self, n_way, k_shot, n_query):
        # Filter classes that have enough examples for this episode
        episode_valid_classes = [
            c for c in self.valid_classes
            if len(self.class_idxs[c]) >= k_shot + n_query
        ]

        if len(episode_valid_classes) < n_way:
            raise ValueError(f"Not enough classes with {k_shot + n_query} examples. "
                            f"Need {n_way} classes but only {len(episode_valid_classes)} available.")

        # Choose random subset of n_way classes
        selected_classes = random.sample(episode_valid_classes, n_way)

        support_indices, query_indices = [], []

        for c in selected_classes:
            indices = self.class_idxs[c]
            if not indices:
                continue
            selected_indices = random.sample(indices, k_shot + n_query)
            support_indices.extend(selected_indices[:k_shot])
            query_indices.extend(selected_indices[k_shot:])

        support_indices = torch.tensor(support_indices, dtype=torch.long)
        query_indices = torch.tensor(query_indices, dtype=torch.long)

        unique_shards = set(self._get_idxs(i)[0] for i in support_indices.tolist() + query_indices.tolist())
        shard_data = {shard_idx: self._load_shard(shard_idx) for shard_idx in unique_shards}

        # Fetch data/labels from those shards
        support_data = torch.cat([shard_data[self._get_idxs(i)[0]]['imgs'][self._get_idxs(i)[1]].unsqueeze(0) for i in support_indices])
        support_label = torch.cat([shard_data[self._get_idxs(i)[0]]['lbls'][self._get_idxs(i)[1]].unsqueeze(0) for i in support_indices])
        query_data = torch.cat([shard_data[self._get_idxs(i)[0]]['imgs'][self._get_idxs(i)[1]].unsqueeze(0) for i in query_indices])
        query_label = torch.cat([shard_data[self._get_idxs(i)[0]]['lbls'][self._get_idxs(i)[1]].unsqueeze(0) for i in query_indices])

        support_data, support_label = batch_augment(support_data, support_label)
        query_data, query_label = batch_augment(query_data, query_label)

        return {
            'support_data': support_data,
            'support_label': support_label,
            'query_data': query_data,
            'query_label': query_label
        }

    def __getitem__(self, idx):
        # Choose random subset of n_way classes
        selected_classes = random.sample(self.valid_classes, self.min_n_way)

        support_indices, query_indices = [], []

        for c in selected_classes:
            indices = self.class_idxs[c]
            if not indices:
                continue
            selected_indices = random.sample(indices, self.try_k_shot + self.try_n_query)
            support_indices.extend(selected_indices[:self.try_k_shot])
            query_indices.extend(selected_indices[self.try_k_shot:])

        support_indices = torch.tensor(support_indices, dtype=torch.long)
        query_indices = torch.tensor(query_indices, dtype=torch.long)

        unique_shards = set(self._get_idxs(i)[0] for i in support_indices.tolist() + query_indices.tolist())
        shard_data = {shard_idx: self._load_shard(shard_idx) for shard_idx in unique_shards}

        # Fetch data/labels from those shards
        support_data = torch.cat([shard_data[self._get_idxs(i)[0]]['imgs'][self._get_idxs(i)[1]].unsqueeze(0) for i in support_indices])
        support_label = torch.cat([shard_data[self._get_idxs(i)[0]]['lbls'][self._get_idxs(i)[1]].unsqueeze(0) for i in support_indices])
        query_data = torch.cat([shard_data[self._get_idxs(i)[0]]['imgs'][self._get_idxs(i)[1]].unsqueeze(0) for i in query_indices])
        query_label = torch.cat([shard_data[self._get_idxs(i)[0]]['lbls'][self._get_idxs(i)[1]].unsqueeze(0) for i in query_indices])

        support_data, support_label = batch_augment(support_data, support_label)
        query_data, query_label = batch_augment(query_data, query_label)

        return {
            'support_data': support_data,
            'support_label': support_label,
            'query_data': query_data,
            'query_label': query_label
        }

    def get_split_info(self):
        """Get information about the current split"""
        total_samples = sum(len(indices) for indices in self.class_idxs if indices)
        samples_per_class = [len(indices) for indices in self.class_idxs if indices]

        return {
            'total_samples': total_samples,
            'valid_classes': len(self.valid_classes),
            'avg_samples_per_class': np.mean(samples_per_class) if samples_per_class else 0,
            'min_samples_per_class': np.min(samples_per_class) if samples_per_class else 0,
            'max_samples_per_class': np.max(samples_per_class) if samples_per_class else 0
        }


In [ ]:
all_pths = list(Path("/content").glob("*.pth"))
all_pths.sort()

meta_path = None
data_paths = []

for pth in all_pths:
    if "meta" in pth.name.lower():
        meta_path = pth
    else:
        data_paths.append(pth)

assert meta_path is not None, "❌ meta.pth file not found!"
meta = torch.load(meta_path)


C:\Users\aniru\AppData\Local\Temp\ipykernel_9252\1539366073.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta = torch.load(meta_path)


In [ ]:
dataset = ShardedEpisodicDataset(data_paths, meta_path, 5, 5, 5, split="train", min_examples_per_class=15)
val_dataset = ShardedEpisodicDataset(data_paths, meta_path, 3, 3, 3, split="val", min_examples_per_class=15)
test_dataset = ShardedEpisodicDataset(data_paths, meta_path, 3, 3, 3, split="test", min_examples_per_class=15)

C:\Users\aniru\AppData\Local\Temp\ipykernel_9252\1519225210.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  meta = torch.load(meta_path)


train_size : 800, test_size : 100, val_size : 100
51 of 200 classes have atleast try_k_shot + try_n_query (10) examples
train_size : 800, test_size : 100, val_size : 100
12 of 200 classes have atleast try_k_shot + try_n_query (6) examples
train_size : 800, test_size : 100, val_size : 100
8 of 200 classes have atleast try_k_shot + try_n_query (6) examples


CNN stuff

In [ ]:
def l2_regularization(model, lambda_l2=0.001):
    l2_norm = 0
    for param in model.parameters():
        l2_norm += param.pow(2).sum()
    return lambda_l2 * l2_norm

In [ ]:
from torchvision.models import resnet34, ResNet34_Weights
import torch.nn.functional as F

class ConvEncoder(nn.Module):
    def __init__(self, input_shape=(1, 128, 1056)):  # C, H, W — use your typical input size
        super().__init__()
        self.input_shape = input_shape
        model = resnet34(weights=ResNet34_Weights.DEFAULT)
        self.features = nn.Sequential(
            model.conv1,
            model.bn1,
            model.relu,
            nn.Dropout(p=0.3),
            model.maxpool,
            model.layer1,
            nn.Dropout(p=0.3),
            model.layer2,
            model.layer3,
            model.layer4
        )
        self.output_channels = 512

        # Compute number of tokens after spatial downsampling
        # Ensure dummy input has 3 channels for ResNet
        dummy_input = torch.zeros((1, 3) + input_shape[1:])
        with torch.no_grad():
            out = self.features(dummy_input)
            _, _, h, w = out.shape
            self.max_nodes = h * w

        # Positional encoding
        self.pos_enc = nn.Parameter(torch.randn(self.max_nodes, self.output_channels) * 0.02)

        self.layer_norm = nn.LayerNorm(self.output_channels)

    def forward(self, x):
      original_shape = x.shape
      if x.shape[1] == 1:
          x = x.repeat(1, 3, 1, 1)  # Convert (B, 1, H, W) → (B, 3, H, W)
      x = self.features(x)  # (B, 512, H/32, W/32)
      x = x.flatten(2).transpose(1, 2)  # (B, N, 512)

      seq_len=x.size(1)
      # Adjust positional encoding
      if x.size(1) != self.pos_enc.size(0):
          pe = F.interpolate(self.pos_enc.unsqueeze(0).transpose(1, 2), size=x.size(1), mode="linear", align_corners=False)
          pe = pe.transpose(1, 2).squeeze(0)
      else:
          pe = self.pos_enc
      x = x + pe
      x = self.layer_norm(x)
      return x

ProtoNet stuff

In [ ]:
def cosine_similarity(x, y):
    """Compute cosine similarity between two tensors"""
    # Normalize the vectors
    x_norm = torch.nn.functional.normalize(x, p=2, dim=1)  # shape: (n, d)
    y_norm = torch.nn.functional.normalize(y, p=2, dim=1)  # shape: (m, d)

    # Compute cosine similarity: x_norm @ y_norm.T
    similarity = torch.mm(x_norm, y_norm.t())  # shape: (n, m)

    return similarity

In [ ]:
class FocalLoss(nn.Module):
  def __init__(self, gamma=2.0, alpha=None, reduction='mean'):
    super().__init__()
    self.gamma = gamma
    self.alpha = alpha
    self.reduction = reduction

  def forward(self, logits, targets):
    bce_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')
    pt = torch.exp(-bce_loss)
    focal_loss = ((1 - pt) ** self.gamma) * bce_loss
    if self.alpha is not None:
      focal_loss = self.alpha * focal_loss
    if self.reduction == 'mean':
      return focal_loss.mean()
    elif self.reduction == 'sum':
      return focal_loss.sum()
    else:
      return focal_loss

In [ ]:
def compute_mAP(y_true, y_scores):
  num_classes = y_true.shape[1]
  APs = []
  for i in range(num_classes):
    if np.sum(y_true[:, i]) >0:
      AP = average_precision_score(y_true[:, i], y_scores[:, i])
      APs.append(AP)
  return np.mean(APs) if len(APs) > 0 else 0.0

In [ ]:
def compute_grad_embeddings(model, data, batch_size):
    embeddings = []
    for i in range(0, len(data), batch_size):
        mini_batch = data[i:i+batch_size]
        embed = model(mini_batch)
        embeddings.append(embed)
    return torch.cat(embeddings, dim=0)

In [ ]:
def prototypical_loss(model, episode, device, use_focal=True, alpha=1.0, gamma=2.0, temperature=0.4,
                      max_subset_size=3, max_num_subsets=500, ema_prototypes=None, ema_momentum=0.25):
    """
    Fixed prototypical loss with temperature scaling to handle distance-based logits properly.

    Args:
        temperature: Scale factor for distances. Try values between 0.1 and 10.0
    """
    model.eval()
    model.to(device)

    support_label = episode['support_label'].to(device)
    query_label = episode['query_label'].to(device)
    support_data = episode['support_data'].to(device)
    query_data = episode['query_data'].to(device)

    def get_power_set(indices, max_len=max_subset_size):
        s = list(indices)
        return list(chain.from_iterable(combinations(s, r) for r in range(1, min(len(s), max_len) + 1)))

    # Gather unique label combinations
    unique_combos = set()
    for lbl in torch.cat([support_label, query_label], 0):
        active = tuple(i for i, v in enumerate(lbl) if v == 1)
        if active:
            unique_combos.add(active)

    all_subsets = set()
    for combo in unique_combos:
        for r in range(1, min(len(combo), max_subset_size)+1):
            for sub in combinations(combo, r):
                all_subsets.add(tuple(sorted(sub)))

    all_subsets = sorted(all_subsets, key=lambda s: (len(s), s))
    if len(all_subsets) > max_num_subsets:
        all_subsets = all_subsets[:max_num_subsets]

    subset_map = {s: i for i, s in enumerate(all_subsets)}

    def extend_labels(lbls):
        ext = torch.zeros((lbls.size(0), len(subset_map)), device=device)
        for i, row in enumerate(lbls):
            active = [j for j,v in enumerate(row) if v==1]
            for r in range(1, min(len(active), max_subset_size)+1):
                for sub in combinations(active, r):
                    idx = subset_map.get(tuple(sorted(sub)))
                    if idx is not None:
                        ext[i, idx] = 1
        return ext

    support_ext = extend_labels(support_label)
    query_ext   = extend_labels(query_label)

    # Get embeddings
    support_emb = compute_grad_embeddings(model, support_data, batch_size=4)
    query_emb   = compute_grad_embeddings(model, query_data, batch_size=4)

    # Vectorized prototype averaging
    w = support_ext.T  # (num_proto, num_support)
    protos_cur = (w @ support_emb) / w.sum(dim=1, keepdim=True).clamp(min=1)

    protos = []
    for i, key in enumerate(subset_map.keys()):
        vec = protos_cur[i]
        if ema_prototypes is not None:
            if key not in ema_prototypes:
                ema_prototypes[key] = vec.clone().detach()
            else:
                ema_prototypes[key] = (
                    (1 - ema_momentum) * ema_prototypes[key] +
                    ema_momentum * vec.detach()
                )
            protos.append(ema_prototypes[key])
        else:
            protos.append(vec)

    protos = torch.stack(protos).detach()

    query_emb = F.normalize(query_emb, dim=1)
    protos = F.normalize(protos, dim=1)

    # CRITICAL FIX: Apply temperature scaling to distances
    dists = torch.matmul(query_emb, protos.T)       # (B, P)

# Optional: scale similarity
    logits = dists / temperature       # <-- This is the key fix!

    # Alternative approaches you can try:
    # Option 1: Normalize embeddings first (cosine similarity)
    # query_emb_norm = F.normalize(query_emb, dim=1)
    # protos_norm = F.normalize(protos, dim=1)
    # logits = torch.mm(query_emb_norm, protos_norm.T)  # Cosine similarity

    # Option 2: Gaussian kernel
    # logits = -dists.pow(2) / (2 * temperature**2)

    # Enhanced diagnostics
    with torch.no_grad():
        print(f"Distance range: [{dists.min().item():.4f}, {dists.max().item():.4f}]")
        print(f"Logits range: [{logits.min().item():.4f}, {logits.max().item():.4f}]")

        probs = torch.sigmoid(logits)
        print(f"Probabilities range: [{probs.min().item():.6f}, {probs.max().item():.6f}]")
        print(f"Prob mean: {probs.mean().item():.6f}")
        print(f"[Train] Avg Logit Range: {logits.min().item()} to {logits.max().item()}")
        print(f"[Train] Avg Prob Mean: {probs.mean().item()}")
        y_true = query_ext.cpu().numpy()
        y_probs = probs.cpu().numpy()

        # Test multiple thresholds
        for thresh in [0.2]:
            y_pred = (y_probs > thresh).astype(int)
            valid_classes = y_true.sum(axis=0) > 0
            if valid_classes.sum() > 0:
                y_true_valid = y_true[:, valid_classes]
                y_pred_valid = y_pred[:, valid_classes]
                f1 = f1_score(y_true_valid, y_pred_valid, average='macro', zero_division=0)
                print(f"F1 at threshold {thresh}: {f1:.4f}")

    # Loss calculation
    if use_focal:
        pos_freq = query_ext.float().mean(0)
        eps = 1e-6
        alpha = pos_freq  # Use positive frequency directly as alpha
        loss_fn = FocalLoss(gamma=gamma, alpha=alpha)
    else:
        # Add class balancing for BCEWithLogitsLoss too
        pos_freq = query_ext.float().mean(0)
        pos_weight = (1 - pos_freq) / (pos_freq + 1e-6)
        pos_weight = torch.clamp(pos_weight, max=10.0)
        loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    loss = loss_fn(logits, query_ext.float())

    # Proper evaluation metrics
    with torch.no_grad():
        probs = torch.sigmoid(logits)
        y_true = query_ext.cpu().numpy()

        # mAP calculation (this should now work properly)
        APs = []
        for i in range(len(subset_map)):
            if y_true[:, i].sum() > 0:  # Only classes with positive samples
                ap = average_precision_score(y_true[:, i], probs[:, i].cpu().numpy())
                APs.append(ap)

        mAP = float(np.mean(APs)) if APs else 0.0
        print(f"Number of valid classes for mAP: {len(APs)}")
        print(f"mAP: {mAP:.4f}")

        # F1 score (fixed - no flattening for multi-label)
        y_pred = (probs > 0.5).cpu().numpy().astype(int)

        # Only compute F1 on classes with positive samples
        valid_classes = y_true.sum(axis=0) > 0
        if valid_classes.sum() > 0:
            f1 = f1_score(y_true[:, valid_classes], y_pred[:, valid_classes],
                         average='macro', zero_division=0)
        else:
            f1 = 0.0

    return loss, f1, mAP

  # # Apply sigmoid and compute loss using BCE
  # #print(distances.shape)
  # #print(query_ext_lbls.float().shape)
  # pos_freq = query_ext_lbls.float().mean(dim=0)
  # epsilon = 1e-6
  # raw_pos_weight = 1.0 / (pos_freq + epsilon)
  # max_weight = 10.0
  # pos_weight  = torch.clamp(raw_pos_weight, max = max_weight)
  # loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
  # loss = loss_fn(distances, query_ext_lbls.float())

  # # compress the extended prediction back to normal
  # pred = torch.zeros(episode['query_data'].shape[0], num_classes, dtype=torch.bool, device=device)
  # for i in range(episode['query_data'].shape[0]):
  #   min_dist = torch.min(distances[i])  # 1. Get smallest distance value for this query
  #   candidates = (distances[i] == min_dist).nonzero(as_tuple=True)[0]  # 2. Find ALL prototypes with this distance
  #   best_idx = max(candidates, key=lambda idx: len(all_subsets[idx]))  # 3. Select largest subset
  #   pred[i, all_subsets[best_idx]] = True

  # true_bool = episode['query_label'].to(device).bool()
  # tp = (pred & true_bool).sum().float()
  # fp = (pred & ~true_bool).sum().float()
  # fn = (~pred & true_bool).sum().float()

  # precision = tp / (tp + fp + 1e-6)  # Adding small value to avoid division by zero
  # recall = tp / (tp + fn + 1e-6)
  # f1_score = 2 * (precision * recall) / (precision + recall + 1e-6)

  # return loss, f1_score

In [ ]:
def predict(model, episode, device):
    model.eval()
    model.to(device)

    num_classes = episode['support_label'].shape[1]
    unique_combinations = set()

    # Collect unique combinations from both support and query
    for label_set in episode['support_label']:
        active = tuple(i for i, val in enumerate(label_set) if val == 1)
        if active:
            unique_combinations.add(active)
    for label_set in episode['query_label']:
        active = tuple(i for i, val in enumerate(label_set) if val == 1)
        if active:
            unique_combinations.add(active)

    def get_power_set(indices):
        return list(chain.from_iterable(combinations(indices, r) for r in range(1, len(indices) + 1)))

    # Generate all subsets
    all_subsets = set()
    for idx_comb in unique_combinations:
        for subset in get_power_set(idx_comb):
            all_subsets.add(tuple(sorted(subset)))
    all_subsets = sorted(all_subsets, key=lambda s: (len(s), s))
    subset_map = {subset: i for i, subset in enumerate(all_subsets)}

    def extend_labels(labels):
        ext = torch.zeros((len(labels), len(all_subsets)), dtype=torch.float, device=device)
        for i, row in enumerate(labels):
            active_indices = tuple(sorted(j for j, val in enumerate(row) if val == 1))
            if active_indices:
                for subset in get_power_set(active_indices):
                    subset = tuple(sorted(subset))
                    ext[i, subset_map[subset]] = 1.0
        return ext

    support_ext_lbls = extend_labels(episode['support_label'])
    query_ext_lbls = extend_labels(episode['query_label'])

    with torch.no_grad():
        support_embeddings = model(episode['support_data'].to(device))
        query_embeddings = model(episode['query_data'].to(device))

    d = support_embeddings.shape[1]
    num_proto = len(all_subsets)
    prototypes = torch.zeros((num_proto, d), device=device)
    counts = torch.zeros(num_proto, device=device)

    # Build prototypes
    for i in range(support_embeddings.shape[0]):
        active_prototypes = support_ext_lbls[i].nonzero(as_tuple=True)[0]
        for p in active_prototypes:
            prototypes[p] += support_embeddings[i]
            counts[p] += 1

    counts = counts.clamp(min=1)
    prototypes /= counts.unsqueeze(1)

    # Debug prototype information
    prototype_norms = torch.norm(prototypes, dim=1)
    query_norms = torch.norm(query_embeddings, dim=1)

    print(f"  Prototype norms - min: {prototype_norms.min():.6f}, max: {prototype_norms.max():.6f}, mean: {prototype_norms.mean():.6f}")
    print(f"  Query norms - min: {query_norms.min():.6f}, max: {query_norms.max():.6f}, mean: {query_norms.mean():.6f}")
    print(f"  Prototypes with zero norm: {(prototype_norms == 0).sum().item()}/{len(prototypes)}")

    # Compute cosine similarity and logits
    similarities = cosine_similarity(query_embeddings, prototypes)

    print(f"  Cosine similarities - min: {similarities.min():.6f}, max: {similarities.max():.6f}, mean: {similarities.mean():.6f}")

    logits = similarities  # cosine similarity is already in [-1, 1] range
    probs_extended = torch.sigmoid(logits)

    print(f"  Logits - min: {logits.min():.6f}, max: {logits.max():.6f}")
    print(f"  Extended probs - min: {probs_extended.min():.6f}, max: {probs_extended.max():.6f}")

    # Convert back to original label space using WEIGHTED AVERAGE approach
    original_probs = torch.zeros((query_embeddings.shape[0], num_classes), device=device)

    # For each query and each class, aggregate probabilities from subsets containing that class
    for i in range(query_embeddings.shape[0]):  # for each query sample
        for j in range(num_classes):  # for each original class
            # Find all extended labels that include this original class
            class_probs = []
            class_weights = []

            for subset_idx, subset in enumerate(all_subsets):
                if j in subset:
                    # Weight by inverse subset size (smaller subsets = more specific = higher weight)
                    weight = 1.0 / len(subset)
                    class_probs.append(probs_extended[i, subset_idx].item())
                    class_weights.append(weight)

            if class_probs:
                # Weighted average instead of max
                weighted_prob = sum(p * w for p, w in zip(class_probs, class_weights)) / sum(class_weights)
                original_probs[i, j] = weighted_prob

    # Generate predictions for original classes
    original_pred = (original_probs > 0.5).int()

    return original_pred, original_probs, episode['query_label'], query_ext_lbls, all_subsets


GAT stuff

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class GATLayer(nn.Module):
    def __init__(self, c_in, c_out, num_heads=8, alpha=0.3, top_k=5, concat_heads = True):
        super().__init__()
        assert c_out % num_heads == 0, "c_out must be divisible by num_heads"
        self.num_heads = num_heads
        self.head_dim = c_out // num_heads
        self.c_out = c_out
        self.top_k = top_k
        self.concat_heads = concat_heads

        self.projection = nn.Linear(c_in, c_out, bias=False)
        self.a = nn.Parameter(torch.Tensor(num_heads, 2 * self.head_dim))
        self.leakyrelu = nn.LeakyReLU(alpha)

        # Add output projection layer
        if concat_heads:
            self.output_project = nn.Linear(c_out, c_out)
        else:
            self.output_project = nn.Linear(self.head_dim, c_out)

        self.layer_norm = nn.LayerNorm(c_out)

        nn.init.xavier_uniform_(self.projection.weight, gain=1.414)
        nn.init.xavier_uniform_(self.a, gain=1.414)
        nn.init.xavier_uniform_(self.output_project.weight, gain=1.414)
        nn.init.zeros_(self.output_project.bias)

    def compute_adj_matrix(self, feats, drop_prob=0.3):
        B, N, _ = feats.shape
        # Normalize features along the last dimension (using lowercase 'normalize')
        norm_feats = F.normalize(feats, p=2, dim=-1)
        # Here, node_feats has shape (batch_size, num_nodes, feature_dim).
        # We need to compute pairwise similarity per batch.
        # Use torch.bmm to perform batched matrix multiplication.
        sim = torch.bmm(norm_feats, norm_feats.transpose(1, 2))  # (batch, num_nodes, num_nodes)
        # Create a binary adjacency matrix based on a threshold (e.g., 0.5)
        if self.top_k is not None:
          topk_values, top_k_indices = torch.topk(sim, k=min(self.top_k+1, N), dim=-1)
          mask = torch.zeros_like(sim)
          mask.scatter_(2, top_k_indices, 1.0)

          eye = torch.eye(N, device=sim.device).unsqueeze(0).expand(B, -1, -1)
          mask = torch.maximum(mask, eye)
          sim = sim * mask

        adj_matrix = (sim > 0.3).float()

        if drop_prob > 0.0:
          eye = torch.eye(N, device=adj_matrix.device).unsqueeze(0).expand(B, -1, -1)
          drop_mask = (torch.rand_like(adj_matrix) > drop_prob).float()
          drop_mask = torch.maximum(drop_mask, eye)
          adj_matrix = adj_matrix * drop_mask

        return adj_matrix

    def forward(self, feats, print_attn_probs=False):
        batch_size, num_nodes, _ = feats.shape

        # Compute the adjacency matrix from node features.
        adj_matrix = self.compute_adj_matrix(feats, drop_prob=0.2)  # Expected shape: (batch_size, num_nodes, num_nodes)

        projected_feats = self.projection(feats)  # (B, N, heads * d_head)
        residual = feats if feats.shape[-1] == self.c_out else None
        node_feats = projected_feats.view(batch_size, num_nodes, self.num_heads, self.head_dim)

        # Now, get the indices of nonzero elements in the adj matrix.
        # Note: For a batched adj_matrix, nonzero() returns indices with shape (num_edges, 3)
        # edges[:, 0] is the batch index, edges[:, 1] is the row index, and edges[:, 2] is the column index.
        edges = (adj_matrix > 0.0).nonzero(as_tuple=False)
        if edges.size(0) == 0:
            # Handle case where no edges exist
            output = projected_feats
            if not self.concat_heads:
                output = node_feats.mean(dim=2)
            output = self.output_project(output)
            output = self.layer_norm(output + residual)
            return output

        batch_indices = edges[:, 0]
        offset = batch_indices * num_nodes
        edges_indices_row = offset + edges[:, 1]
        edges_indices_col = offset + edges[:, 2]

        node_feats_flat = node_feats.view(batch_size * num_nodes, self.num_heads, self.head_dim)

        a_input = torch.cat([
            torch.index_select(node_feats_flat, dim=0, index=edges_indices_row),
            torch.index_select(node_feats_flat, dim=0, index=edges_indices_col)
        ], dim=-1)

        attn_logits = torch.einsum('ehd,hd->eh', a_input, self.a)
        attn_logits = self.leakyrelu(attn_logits)


        # Create an attention matrix with shape (batch_size, num_nodes, num_nodes, num_heads)
        attn_matrix = torch.full((batch_size, num_nodes, num_nodes, self.num_heads), -1e9, device=node_feats.device, dtype=node_feats.dtype)
        # Assign computed logits to positions where adj_matrix is 1.
        # We expand adj_matrix to have a head dimension.
        for i, (b, r, c) in enumerate(edges):
            attn_matrix[b, r, c, :] = attn_logits[i, :]

        attn_probs = F.softmax(attn_matrix, dim=2)

        if self.training:
            attn_probs = self.dropout_layer(attn_probs)

        if print_attn_probs:
            print("Attention probabilities shape:", attn_probs.shape)
            print("Sample attention probs:\n", attn_probs[0, :5, :5, 0].detach().cpu())

        aggr_feats = torch.einsum('bnmh,bmhd->bnhd', attn_probs, node_feats)

        if self.concat_heads:
            output = aggr_feats.reshape(batch_size, num_nodes, -1)
        else:
            output = aggr_feats.mean(dim=2)

        output = self.output_project(output)

        if residual is not None and output.shape == residual.shape:
            output = self.layer_norm(output + residual)

        return output

Combining all the models

In [ ]:
class CnnGat(nn.Module):
    def __init__(self, cnn_encoder, gat_layer, per_node_dim, device):
        super().__init__()
        self.device = device

        # 1) Encoder on CPU initially
        self.cnn_encoder = cnn_encoder()

        # 2) Compute N on CPU
        with torch.no_grad():
            dummy = torch.zeros(1, 1, 128, 1056)   # **no .to(device) here**
            N = self.cnn_encoder(dummy).size(1)    # gets number of nodes

        # 3) GAT on CPU too
        self.gat_layer = gat_layer(
            c_in=self.cnn_encoder.output_channels,
            c_out=per_node_dim
        )
        self.norm = nn.LayerNorm(N * per_node_dim)


    def forward(self, data):
        small_bs = 4
        B = data.size(0)
        encoded = []

        # 1) Batch through encoder to avoid OOM
        for i in range(0, B, small_bs):
            chunk = data[i:i+small_bs].to(self.device)
            with torch.no_grad():
                out = self.cnn_encoder(chunk)  # <-- (miniB, N, D)
            encoded.append(out)
        # 2) concatenate back into (B, N, D)
        node_feats = torch.cat(encoded, dim=0)  # (B, N, D)
        # 3) GAT over nodes → (B, N, d'), where N = nodes per sample
        gat_out = self.gat_layer(node_feats)    # (B, N, d')
        # 4) flatten to (B, N * d') = (B, embed_size)
        flat = gat_out.reshape(B, -1)           # (B, embed_size)
        # 5) optional layer norm
        return self.norm(flat)


Training Stuff

In [ ]:
from tqdm.auto import tqdm
import torch.optim.lr_scheduler as lr_scheduler
import random

ema_prototypes = {}
def check_val(model, val_dataset, ema_prototypes, num_checks=10):
    model.eval()  # Set to evaluation mode

    running_f1 = 0.0
    min_f1 = float('inf')
    max_f1 = float('-inf')

    running_map = 0.0
    min_map = float('inf')  # Add min/max tracking for mAP
    max_map = float('-inf')

    with torch.no_grad():  # Disable gradient computation
        for i in range(num_checks):
            test_ep = val_dataset[0]
            _, f1, mAP = prototypical_loss(model, test_ep, device, use_focal=True, gamma=2.0, alpha=1.0, ema_prototypes=ema_prototypes, ema_momentum=0.25)

            f1_val = f1.item() if torch.is_tensor(f1) else f1
            mAP_val = mAP.item() if torch.is_tensor(mAP) else mAP

            running_f1 += f1_val
            running_map += mAP_val

            min_f1 = min(min_f1, f1_val)
            max_f1 = max(max_f1, f1_val)
            min_map = min(min_map, mAP_val)  # Track mAP min/max
            max_map = max(max_map, mAP_val)

    model.train()  # Switch back to training mode

    avg_f1 = running_f1 / num_checks
    avg_map = running_map / num_checks

    print(f"\nVal: Avg F1: {avg_f1:.2f}, min: {min_f1:.2f}, max: {max_f1:.2f}")
    print(f"Val: Avg mAP: {avg_map:.2f}, min: {min_map:.2f}, max: {max_map:.2f}")

Main

In [ ]:
def train(model, dataset, val_dataset, optimizer, batch_size, epochs, device):
  model.train()
  model.to(device)

  random.seed(42)
  torch.manual_seed(42)

  scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-5)

  for epoch in range(epochs):
    running_loss = 0.0
    running_f1 = 0.0
    running_mAP = 0.0
    for batch in tqdm(range(batch_size)):
      optimizer.zero_grad()
      episode = dataset[0] # the indexing on our custom dataset is nonsense on purpose, every index gives random result each time

      loss, f1, mAP = prototypical_loss(model, episode, device, use_focal=True, gamma=2.0, alpha=1.0, ema_prototypes=ema_prototypes, ema_momentum=0.25)
      loss += l2_regularization(model, lambda_l2=1e-5)
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
      optimizer.step()

      running_loss += loss.item()
      running_f1 += f1.item() if torch.is_tensor(f1) else f1
      running_mAP += mAP.item() if torch.is_tensor(mAP) else mAP

      del episode, loss, f1
      gc.collect()
      torch.cuda.empty_cache()

    epoch_loss = running_loss / batch_size
    epoch_f1 = running_f1 / batch_size
    epoch_mAP = running_mAP / batch_size
    print(f'\nEpoch {epoch+1} Complete - Loss: {epoch_loss:.4f} F1: {epoch_f1:.2f} mAP: {epoch_mAP:.2f}\n')
    check_val(model, val_dataset, ema_prototypes)
    scheduler.step()

In [ ]:

model = CnnGat(ConvEncoder, GATLayer, per_node_dim=64, device=device)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr = 3e-4, weight_decay=1e-4)

train(model, dataset, val_dataset, optimizer, batch_size=4, epochs=25, device=device)

c:\Users\aniru\.conda\envs\pytorch-env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1 Complete - Loss: 4502.7261 F1: 0.00 mAP: 0.05
Val: Avg F1: 0.00, min: 0.00, max: 0.00, Avg mAP: 0.16


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 2 Complete - Loss: 3707.6733 F1: 0.00 mAP: 0.06
Val: Avg F1: 0.00, min: 0.00, max: 0.00, Avg mAP: 0.17


...

Epoch 48 Complete - Loss: 36.1054 F1: 0.00 mAP: 0.49
Val: Avg F1: 0.00, min: 0.00, max: 0.00, Avg mAP: 0.38


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 49 Complete - Loss: 33.8246 F1: 0.00 mAP: 0.62
Val: Avg F1: 0.00, min: 0.00, max: 0.00, Avg mAP: 0.34


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 50 Complete - Loss: 40.6560 F1: 0.00 mAP: 0.70
Val: Avg F1: 0.00, min: 0.00, max: 0.00, Avg mAP: 0.41


In [19]:
# Testing
import matplotlib.pyplot as plt

In [ ]:
test_episode = test_dataset[0]

support_data = test_episode['support_data']
support_label = test_episode['support_label']
query_data = test_episode['query_data']
query_label = test_episode['query_label']

# print shape for debugging
print(f"{test_episode['support_data'].shape}, {test_episode['support_label'].shape}, {test_episode['query_data'].shape}, {test_episode['query_label'].shape}")

C:\Users\aniru\AppData\Local\Temp\ipykernel_9252\1519225210.py:83: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  shard = torch.load(shard_path)


torch.Size([9, 3, 224, 224]), torch.Size([9, 200]), torch.Size([9, 3, 224, 224]), torch.Size([9, 200])


In [ ]:
# prepare data for visualization
imgs = test_episode['query_data'].cpu().numpy().transpose(0, 2, 3, 1)
col_names = ['Accelerating_and_revving_and_vroom', 'Accordion', 'Acoustic_guitar', 'Aircraft', 'Alarm', 'Animal', 'Applause', 'Bark', 'Bass_drum', 'Bass_guitar', 'Bathtub_(filling_or_washing)', 'Bell', 'Bicycle', 'Bicycle_bell', 'Bird', 'Bird_vocalization_and_bird_call_and_bird_song', 'Boat_and_Water_vehicle', 'Boiling', 'Boom', 'Bowed_string_instrument', 'Brass_instrument', 'Breathing', 'Burping_and_eructation', 'Bus', 'Buzz', 'Camera', 'Car', 'Car_passing_by', 'Cat', 'Chatter', 'Cheering', 'Chewing_and_mastication', 'Chicken_and_rooster', 'Child_speech_and_kid_speaking', 'Chime', 'Chink_and_clink', 'Chirp_and_tweet', 'Chuckle_and_chortle', 'Church_bell', 'Clapping', 'Clock', 'Coin_(dropping)', 'Computer_keyboard', 'Conversation', 'Cough', 'Cowbell', 'Crack', 'Crackle', 'Crash_cymbal', 'Cricket', 'Crow', 'Crowd', 'Crumpling_and_crinkling', 'Crushing', 'Crying_and_sobbing', 'Cupboard_open_or_close', 'Cutlery_and_silverware', 'Cymbal', 'Dishes_and_pots_and_pans', 'Dog', 'Domestic_animals_and_pets', 'Domestic_sounds_and_home_sounds', 'Door', 'Doorbell', 'Drawer_open_or_close', 'Drill', 'Drip', 'Drum', 'Drum_kit', 'Electric_guitar', 'Engine', 'Engine_starting', 'Explosion', 'Fart', 'Female_singing', 'Female_speech_and_woman_speaking', 'Fill_(with_liquid)', 'Finger_snapping', 'Fire', 'Fireworks', 'Fixed-wing_aircraft_and_airplane', 'Fowl', 'Frog', 'Frying_(food)', 'Gasp', 'Giggle', 'Glass', 'Glockenspiel', 'Gong', 'Growling', 'Guitar', 'Gull_and_seagull', 'Gunshot_and_gunfire', 'Gurgling', 'Hammer', 'Hands', 'Harmonica', 'Harp', 'Hi-hat', 'Hiss', 'Human_group_actions', 'Human_voice', 'Idling', 'Insect', 'Keyboard_(musical)', 'Keys_jangling', 'Knock', 'Laughter', 'Liquid', 'Livestock_and_farm_animals_and_working_animals', 'Male_singing', 'Male_speech_and_man_speaking', 'Mallet_percussion', 'Marimba_and_xylophone', 'Mechanical_fan', 'Mechanisms', 'Meow', 'Microwave_oven', 'Motor_vehicle_(road)', 'Motorcycle', 'Music', 'Musical_instrument', 'Ocean', 'Organ', 'Packing_tape_and_duct_tape', 'Percussion', 'Piano', 'Plucked_string_instrument', 'Pour', 'Power_tool', 'Printer', 'Purr', 'Race_car_and_auto_racing', 'Rail_transport', 'Rain', 'Raindrop', 'Ratchet_and_pawl', 'Rattle', 'Rattle_(instrument)', 'Respiratory_sounds', 'Ringtone', 'Run', 'Sawing', 'Scissors', 'Scratching_(performance_technique)', 'Screaming', 'Screech', 'Shatter', 'Shout', 'Sigh', 'Singing', 'Sink_(filling_or_washing)', 'Siren', 'Skateboard', 'Slam', 'Sliding_door', 'Snare_drum', 'Sneeze', 'Speech', 'Speech_synthesizer', 'Splash_and_splatter', 'Squeak', 'Stream', 'Strum', 'Subway_and_metro_and_underground', 'Tabla', 'Tambourine', 'Tap', 'Tearing', 'Telephone', 'Thump_and_thud', 'Thunder', 'Thunderstorm', 'Tick', 'Tick-tock', 'Toilet_flush', 'Tools', 'Traffic_noise_and_roadway_noise', 'Train', 'Trickle_and_dribble', 'Truck', 'Trumpet', 'Typewriter', 'Typing', 'Vehicle', 'Vehicle_horn_and_car_horn_and_honking', 'Walk_and_footsteps', 'Water', 'Water_tap_and_faucet', 'Waves_and_surf', 'Whispering', 'Whoosh_and_swoosh_and_swish', 'Wild_animals', 'Wind', 'Wind_chime', 'Wind_instrument_and_woodwind_instrument', 'Wood', 'Writing', 'Yell', 'Zipper_(clothing)']

In [ ]:
# prepare grid
grid_size = int(np.ceil(np.sqrt(imgs.shape[0])))
fig, axes = plt.subplots(grid_size, grid_size, figsize=(12, 12))
axes = axes.flatten()

for i in range(imgs.shape[0]):
    ax = axes[i]
    img = imgs[i]
    ax.imshow(img)
    ax.axis("off")
    active_indices = torch.nonzero(pred[i], as_tuple=True)[0]
    selected_labels = [col_names[i] for i in active_indices]

    # get query labels
    query_labels = torch.nonzero(test_episode['query_label'][i], as_tuple=True)[0]
    query_label_names = [col_names[i] for i in query_labels]

    ax.set_title(f"Pred: {', '.join(selected_labels)}\nTrue: {', '.join(query_label_names)}", fontsize=7)

for j in range(i+1, len(axes)):
    axes[j].axis("off")

plt.tight_layout()
plt.show()

In [ ]:
checks = 10

running_f1 = 0.0
running_map = 0.0
min_f1 = 1.0
max_f1 = 0.0
min_map = 1.0
max_map = 0.0

    # Debug counters
zero_pred_count = 0
zero_true_count = 0

for i in tqdm(range(checks)):
    test_ep = test_dataset.getEpisode(6, 6, 6)
    pred, probs, true_labels, ext_true_labels, all_subsets = predict(model, test_ep, device)

        # Convert to numpy for sklearn metrics
    true_labels_np = true_labels.cpu().numpy()
    pred_np = pred.cpu().numpy()
    probs_np = probs.cpu().numpy()

        # Debug information
    num_pred_positives = pred_np.sum()
    num_true_positives = true_labels_np.sum()

    if num_pred_positives == 0:
            zero_pred_count += 1
    if num_true_positives == 0:
            zero_true_count += 1

        # Print debug info for first few episodes
    if i < 3:
        print(f"\nEpisode {i+1} Debug:")
        print(f"  Pred shape: {pred_np.shape}, True shape: {true_labels_np.shape}")
        print(f"  Predicted positives: {num_pred_positives}")
        print(f"  True positives: {num_true_positives}")
        print(f"  Prob range: [{probs_np.min():.4f}, {probs_np.max():.4f}]")
        print(f"  Number of subsets: {len(all_subsets)}")

            # Show some sample predictions vs true labels
        print(f"  Sample predictions (first 5 queries, first 10 classes):")
        for q in range(min(5, pred_np.shape[0])):
            print(f"    Query {q}: pred={pred_np[q][:10]}, true={true_labels_np[q][:10]}")

        # Compute metrics
    f1 = f1_score(true_labels_np, pred_np, average='macro', zero_division=0)
    map_value = compute_mAP(true_labels_np, probs_np)

        # Update running statistics
    running_f1 += f1
    min_f1 = min(min_f1, f1)
    max_f1 = max(max_f1, f1)

    running_map += map_value
    min_map = min(min_map, map_value)
    max_map = max(max_map, map_value)

    # Calculate averages
avg_f1 = running_f1 / checks
avg_map = running_map / checks

print(f"\n=== EVALUATION RESULTS ===")
print(f"Avg F1 in test set: {avg_f1:.4f}, min: {min_f1:.4f}, max: {max_f1:.4f}")
print(f"Avg mAP in test set: {avg_map:.4f}, min: {min_map:.4f}, max: {max_map:.4f}")
print(f"\n=== DEBUG INFO ===")
print(f"Episodes with zero predictions: {zero_pred_count}/{checks}")
print(f"Episodes with zero true labels: {zero_true_count}/{checks}")
